## Notebook for performing QTS analysis for PD GRS and expression

In [ ]:
!date

#### import libraries and set notebook variables

In [ ]:
import pandas as pd
import os
import numpy as np
import statsmodels.stats.multitest as smm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# parameters
cohort = 'foundin'
day = 'da65'

In [ ]:
# naming
cohort_build = f'{cohort}.{day}'
quant_type = 'atac'

# directories
home_dir = '/home/gibbsr'
wrk_dir = f'{home_dir}/{cohort}/caqtl'
quant_dir = f'{wrk_dir}/quants'
info_dir = f'{wrk_dir}/sample_info'
qts_dir = f'{wrk_dir}/qts'

# in files
covs_file = f'{info_dir}/{cohort}_{quant_type}_sample_info.csv'
grs_file = f'{info_dir}/{cohort}.grs.scaled.csv'
quants_file = f'{quant_dir}/{cohort_build}.scaled.adj.hdf5'

# out files
qts_file = f'{qts_dir}/{cohort_build}.qts.csv'

# constants

In [ ]:
os.makedirs(qts_dir, exist_ok=True)

#### analysis functions

In [ ]:
# functions to run the linear regression
def reg_model(y, x, term):
    Xb = sm.add_constant(x)
    reg_model = sm.OLS(y, Xb).fit()

# #     formula api style
#     this_formula = f'Q("{this_trait}") ~ GRS + predicted_Lymphocytes + GRS * predicted_Lymphocytes'
#     reg_model = smf.ols(this_formula, data=this_quants).fit()

    # return the coef, stderr, adjusted r2, number of terms, and p-value
    return [reg_model.params[term], reg_model.bse[term], reg_model.rsquared_adj, 
            reg_model.params.shape[0], reg_model.pvalues[term]]


def grsresgression(cov_df, traits_df, dep_term='', extra_dep_terms=None):
    print(dep_term)
    this_sample = set(cov_df.index) & set(traits_df.index)
    print(len(this_sample))
    
    this_cov_df = cov_df.loc[cov_df.index.isin(this_sample)]
    this_traits_df = traits_df.loc[traits_df.index.isin(this_sample)]
    this_cov_df = this_cov_df.reindex(this_traits_df.index)
    
    if not extra_dep_terms is None:
        this_cov_df = this_cov_df[[dep_term] + extra_dep_terms]
    else:
        this_cov_df = this_cov_df[[dep_term]]
    
    lm_results = this_traits_df.apply(lambda x: reg_model(x, this_cov_df, dep_term),
                                      result_type='expand').\
    rename(index={0: 'coef', 1: 'stderr', 2: 'r2_adj', 3:'term_cnt', 4:'p-value'}).T

    alpha=0.05
    method='fdr_bh'
    test_adjust = smm.multipletests(np.array(lm_results['p-value']),
                                    alpha=alpha, method=method)
    lm_results['bh_fdr'] = test_adjust[1]
    return lm_results

In [ ]:
#plot the QTS
def plotqts(trait_id, study_name, score_df, traits_df):
    this_scores_df = score_df.loc[score_df.index.isin(traits_df.index)]
    this_scores_df = this_scores_df.reindex(traits_df.index)

    temp = traits_df.merge(this_scores_df,left_index=True,right_index=True)

    plt.figure(figsize=(9, 9))
    sns.regplot(x=trait_id,y='GRS', data=temp, ci=95)
    sns.scatterplot(x=trait_id,y='GRS', data=temp, hue='DX')
    plt.xlabel('Trait')
    plt.ylabel('GRS')
    plt.title(f'{trait_id} in {study_name}')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0,prop={'size': 10})
    plt.show()

#### load the known sample covariates

In [ ]:
covs_df = pd.read_csv(covs_file, index_col=0)
print(covs_df.shape)
# if any duplicates, keep first
covs_df = covs_df[~covs_df.index.duplicated(keep='first')]
# display(covs_df.head())

#### load and add GRS

In [ ]:
grs_df = pd.read_csv(grs_file)
print(grs_df.shape)
# display(grs_df.head())

In [ ]:
# covs_df = covs_df.merge(grs_df, how='inner', left_on='wgsid', right_on='IID')
covs_df = covs_df.reset_index().merge(grs_df, how='inner', left_on='wgsid', 
                                      right_on='IID').set_index('assayid')
print(covs_df.shape)
# display(covs_df.head())

#### load the expression data

In [ ]:
%%time
quants_df = pd.read_hdf(quants_file, index_col=0)
print(quants_df.shape)
# display(quants_df.head())

In [ ]:
set(quants_df.index) - set(covs_df.index)

#### run the QTS (feature ~ GRS regression)

In [ ]:
%%time
grs_results = grsresgression(covs_df, quants_df, dep_term='GRS')

#### save the results files

In [ ]:
grs_results.to_csv(qts_file)

In [ ]:
print(grs_results.shape)
print(grs_results.loc[grs_results['bh_fdr'] <= 0.05].shape)

In [ ]:
grs_results.describe()

In [ ]:
display(grs_results.loc[grs_results['bh_fdr'] <= 0.05])

#### plot the most significant results

In [ ]:
scaled_quants_df =  pd.DataFrame(data=MinMaxScaler().fit_transform(quants_df), 
                               index=quants_df.index, columns=quants_df.columns)
print(scaled_quants_df.shape)
# display(scaled_quants_df.head())

In [ ]:
this_trait = grs_results.loc[grs_results['bh_fdr'] == min(grs_results['bh_fdr']),].index[0]
print(grs_results.loc[this_trait])
plotqts(this_trait, cohort.upper(),  covs_df, quants_df)

In [ ]:
plotqts(this_trait, cohort.upper(), covs_df, scaled_quants_df)

#### now run the model with the cell fractions as covariates

In [ ]:
covs_df['DopaminergicNeurons'].fillna(covs_df['DopaminergicNeurons'].mean(), inplace=True)
covs_df['EarlyneuronProgenitor'].fillna(covs_df['EarlyneuronProgenitor'].mean(), inplace=True)
covs_df['ImmatureDopaminergicNeurons'].fillna(covs_df['ImmatureDopaminergicNeurons'].mean(), inplace=True)
covs_df['ProliferatingFloorPlateProgenitors'].fillna(covs_df['ProliferatingFloorPlateProgenitors'].mean(), inplace=True)
covs_df['LateneuronProgenitor'].fillna(covs_df['LateneuronProgenitor'].mean(), inplace=True)
covs_df['Ependymal-likeCells'].fillna(covs_df['Ependymal-likeCells'].mean(), inplace=True)
covs_df['Neuroepithelial-likeCells'].fillna(covs_df['Neuroepithelial-likeCells'].mean(), inplace=True)

In [ ]:
%%time
# cell_cols = ['DopaminergicNeurons']
cell_cols = ['DopaminergicNeurons', 'EarlyneuronProgenitor',
             'ImmatureDopaminergicNeurons', 'ProliferatingFloorPlateProgenitors',
             'LateneuronProgenitor', 'Ependymal-likeCells',
             'Neuroepithelial-likeCells']
# cell_cols = ['age_at_baseline'] + cell_cols
grs_results = grsresgression(covs_df, quants_df, dep_term='GRS', extra_dep_terms=cell_cols)

In [ ]:
grs_results.describe()

In [ ]:
print(grs_results.shape)
print(grs_results.loc[grs_results['bh_fdr'] <= 0.05].shape)

In [ ]:
display(grs_results.loc[grs_results['bh_fdr'] <= 0.05])

#### plot the most significant results

In [ ]:
this_trait = grs_results.loc[grs_results['bh_fdr'] == min(grs_results['bh_fdr']),].index[0]
print(grs_results.loc[this_trait])
plotqts(this_trait, cohort.upper(),  covs_df, quants_df)

In [ ]:
plotqts(this_trait, cohort.upper(), covs_df, scaled_quants_df)